In [ ]:
#require "core";;
#require "yaml.unix";;
#require "yaml";;
#require "fpath"
#require "yaml.unix"
#require "lwt"
#require "cohttp"
#require "cohttp-lwt-unix"

In [7]:
open Lwt
open Cohttp
open Cohttp_lwt_unix

let keyvallist_to_form_urlencoded p =
  p |> List.map (fun (k,v) -> Printf.sprintf "%s=%s" k v)
  |> String.concat "&"
  |> Cohttp_lwt.Body.of_string

let keyvallist_to_form_json p = p
  |> List.map (fun (k,v) -> Printf.sprintf {|"%s": "%s"|} k v)
  |> String.concat ", "
  |> Printf.sprintf "{%s}"
  |> Cohttp_lwt.Body.of_string

let test_post =
    let uri = Uri.of_string "https://httpbin.org/post" in
    (* let headers = Cohttp.Header.init_with "Bearer" token in *)
    let body = keyvallist_to_form_json [("mykey", "myvalue")] in
    Cohttp_lwt_unix.Client.post ~body uri >>= fun (resp, body) ->
      let code = resp |> Response.status |> Code.code_of_status in
      Printf.printf "Response code: %d\n" code;
      Printf.printf "Headers: %s\n" (resp |> Response.headers |> Header.to_string);
      body |> Cohttp_lwt.Body.to_string >|= fun body ->
      Printf.printf "Body of length: %d\n" (String.length body);
      body

let () =
  let respBody = Lwt_main.run test_post in
  print_endline (respBody)

val keyvallist_to_form_urlencoded :
  (string * string) list -> Cohttp_lwt.Body.t = <fun>


val keyvallist_to_form_json : (string * string) list -> Cohttp_lwt.Body.t =
  <fun>


val test_post : string Lwt.t = <abstr>


Response code: 200
Headers: access-control-allow-credentials: true
access-control-allow-origin: *
connection: keep-alive
content-length: 381
content-type: application/json
date: Thu, 04 Feb 2021 23:54:53 GMT
server: gunicorn/19.9.0


Body of length: 381
{
  "args": {}, 
  "data": "{\"mykey\": \"myvalue\"}", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Content-Length": "20", 
    "Host": "httpbin.org", 
    "User-Agent": "ocaml-cohttp/2.5.4", 
    "X-Amzn-Trace-Id": "Root=1-601c894d-2efc115d6e7242e960db46e4"
  }, 
  "json": {
    "mykey": "myvalue"
  }, 
  "origin": "100.17.36.224", 
  "url": "https://httpbin.org/post"
}

